In [ ]:
import reproducibility

In [ ]:
# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan
from data_funcs import raws_data, synthetic_data, plot_data, check_data, mse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict

meso_token="b40cb52cbdef43ef81329b84e8fd874f"


In [ ]:
# Change directory for data read/write
os.chdir('data')

## Retrieve Data

In [ ]:
# read test datasets
# case_data = from_json('rnn_orig.json')
filename = 'testing_dict.pickle'
with open(filename, 'rb') as handle:
    test_dict = pickle.load(handle)
    for case in test_dict:
        test_dict[case]['case'] = case
        test_dict[case]['filename'] = filename


In [ ]:
case = 'case11'    # choose a test dataset

In [ ]:
raws_dat = test_dict[case]
case_data=raws_dat
check_data(case_data)
hours=raws_dat['hours']
h2=raws_dat['h2'] 

In [ ]:
%matplotlib inline
plot_data(raws_dat)

In [ ]:
plot_data(raws_dat,hmax=h2)

In [ ]:
%matplotlib inline
if 0:
    synt_dat=synthetic_data()  # just testinh
    check_data(raws_dat)
    plot_data(synt_dat)

In [ ]:
# dictionary raws_dat has all that is needed for the run 
# keeping the name raws_dat for now even if it may not be raws data



## Fit Augmented KF

In [ ]:
m,Ec = mod.run_augmented_kf(raws_dat)  # extract from state
raws_dat['m']=m
raws_dat['Ec']=Ec
plot_data(raws_dat,title2='augmented KF')

In [ ]:
plot_data(raws_dat,hmin=0,hmax=h2,title2='augmented KF')

In [ ]:
plot_data(raws_dat,hmin=h2,hmax=hours,title2='augmented KF')

In [ ]:
mse_data(raws_dat) 


## Fit RNN Model

In [ ]:
# Set seed for reproducibility
reproducibility.set_seed()

In [ ]:
plot_data(case_data,title2='from testing_dict.pickle',hmin=0,hmax=h2)

In [ ]:
plot_data(case_data,title2='from testing_dict.pickle',hmin=h2,hmax=hours)

In [ ]:
if 'm' in case_data:
    mse_data(case_data)  # just check sdolution if there
    del case_data['m']   # cleanup - remove old solution if any

In [ ]:
verbose = False
# Set seed for reproducibility
reproducibility.set_seed()
rnn_dat = create_rnn_data(case_data,scale=False, verbose=verbose)

In [ ]:
## Check 1: equilibrium input data the same

print(hash2(rnn_dat['Et']))
print(hash2(rnn_dat['x_train']))
print(hash2(rnn_dat['y_train']))

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    rnn_dat['hours'],
    activation=['linear','linear'],
    hidden_units=6,
    dense_units=1,
    dense_layers=1,
    verbose = verbose
)

In [ ]:
m = rnn_predict(model_predict, rnn_dat, rnn_dat['hours'], verbose = verbose)
case_data['m'] = m
note = 'm replaced by a solution from fmda_rnn_rain'
if 'note' in case_data:
    case_data['note'] = case_data['note'] + '\n' + note
else:
    case_data['note'] = note
check_data(case_data)

In [ ]:
plot_data(case_data,title2='with trained RNN',hmin=0,hmax=600)


In [ ]:
mse_data(case_data)

In [ ]:
plot_data(case_data,title2='RNN prediction',hmin=300,hmax=600)

In [ ]:
print(hash2(case_data['m']))
print(hash2(model_predict.get_weights()))
print(model_predict.get_weights())

---
---

<mark>Start Here after Check 1<\mark>

In [ ]:
reproducibility.set_seed()

In [ ]:
from utils import vprint

samples = rnn_dat['samples']
features = rnn_dat['features']
timesteps = rnn_dat['timesteps']
    
model_fit=create_RNN_2(hidden_units=6, 
                        dense_units=1, 
                        batch_shape=(samples,timesteps,features),
                        stateful=True,
                        return_sequences=False,
                        # initial_state=h0,
                        activation=['linear','linear'],
                        dense_layers=1)

from keras.utils.vis_utils import plot_model
plot_model(model_fit, to_file='model_plot.png', 
           show_shapes=True, show_layer_names=True)

In [ ]:
## Check 2: Untrained RNN initialized with same weights

hash2(model_fit.get_weights())

In [ ]:
Et = rnn_dat['Et']
model_predict=create_RNN_2(hidden_units=6, dense_units=1,  
                            input_shape=(hours,features),stateful = False,
                            return_sequences=True,
                            activation=['linear','linear'],dense_layers=1)

In [ ]:
## Check 3: Second model initialization same weights

hash2(model_predict.get_weights())

In [ ]:
print(rnn_dat)
x_train = rnn_dat['x_train']
y_train = rnn_dat['y_train']
type(x_train)

# print dimensions and set initial state
print('model_fit input shape',x_train.shape,'output shape',model_fit(x_train).shape)

# fitting
DeltaE = 0
w_exact=  [np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.*DeltaE])]
    
w_initial=[np.array([[1.-np.exp(-0.1)]]), 
           np.array([[np.exp(-0.1)]]), 
           np.array([0.]),
           np.array([[1.0]]),
           np.array([-1.0])]
r = 1e-2
r = 0.
print('randomization of initial weights ',r)
w=model_fit.get_weights()
for i in range(len(w)):
    vprint('weight',i,'shape',w[i].shape,'ndim',w[i].ndim,'given',w_initial[i].shape)
    for j in range(w[i].shape[0]):
        if w[i].ndim==2:
            for k in range(w[i].shape[1]):
                w[i][j][k]=w_initial[i][0][0]/w[i].shape[0] + np.random.normal(0,1)*r
        else:
            w[i][j]=w_initial[i][0] + np.random.normal(0,1)*r
model_fit.set_weights(w)

In [ ]:
## Check 4: weights and inputs the same after this step 

print(hash2(model_fit.get_weights()))
print(hash2(x_train))
print(hash2(y_train))

In [ ]:
print('model_fit input shape',x_train.shape,'output shape',y_train.shape)
print('x_train',x_train)
print('y_train',y_train)

In [ ]:
reproducibility.set_seed()

In [ ]:
model_fit.get_weights()

In [ ]:
model_fit.fit(x_train, y_train, epochs=5000, verbose=0, batch_size=samples)
w_fitted=model_fit.get_weights()
for i in range(len(w)):
    vprint('weight',i,' exact:',w_exact[i],':  initial:',w_initial[i],' fitted:',w_fitted[i])
    
model_predict.set_weights(w_fitted)

In [ ]:
## Check 5: Weights NOT the same after fitting

hash2(model_fit.get_weights())

In [ ]:
model_fit.get_weights()

In [ ]:
model_fit.get_config()

In [ ]:
# evaluate model
model_predict.set_weights(w_fitted)
x_input=np.reshape(Et,(1, hours, 2))
y_output = model_predict.predict(x_input)
print('x_input.shape=',x_input.shape,'y_output.shape=',y_output.shape)
# print(shift)
m = np.reshape(y_output,hours)
fitted_data = case_data.copy()
fitted_data.update({'m':m,'title':"First RNN forecast"})
del fitted_data['Ec']

In [ ]:
mse_data(fitted_data)
plot_data(fitted_data,title2='RNN fitted')